In [1]:
from src.graph import *
from src.mdp import *
from src.policy import *
from src.nyc import *

/Users/fabiopruneri/Documents/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [2]:
# Simulating the MDP over a finite number of steps
#  for a given policy (approximate policy evaluation)

mdp = MDP['small']

##### policy_random

In [3]:
# Pick a taxi driver at random, including those that might
#  be far away or already completing a query
mdp.eval(policy_random, no_iter=10000, verbose_result=True)

Total reward over 10000 steps:  -22982
Approx 229.8  pct of avg trip time


-22982

##### policy_earliest_free

In [4]:
# Pick a random taxi driver which is free
#  (or has the least time remaning before completing current ride)
mdp.eval(policy_earliest_free, no_iter=10000, verbose_result=True)

Total reward over 10000 steps:  -10296
Approx 103.0  pct of avg trip time


-10296

##### policy_greedy

In [5]:
# Greedy policy: pick taxi driver that minimizes immediate cost,
#  so minimizes distance from start point + time to complete current query (if busy)
mdp.eval(policy_greedy(mdp), no_iter=10000, verbose_result=True)

Total reward over 10000 steps:  -5875
Approx 58.8  pct of avg trip time


-5875

# Table of results for all graphs

In [2]:
# Results are in percentage of average trip time
# So a value of 20 would mean the customer waits on average 20% of the length of the trip

In [13]:
# Takes 1 minute to run
for name, mdp in MDP.items():
    print("MDP ", name)
    print("   policy_random:       ", round(mdp.eval(policy_random, no_iter=100000, return_pct_trip=True)))
    print("   policy_earliest_free:", round(mdp.eval(policy_earliest_free, no_iter=100000, return_pct_trip=True)))
    print("   policy_greedy:       ", round(mdp.eval(policy_greedy(mdp), no_iter=100000, return_pct_trip=True)))
    print("\n")

MDP  small
   policy_random:        228
   policy_earliest_free: 102
   policy_greedy:        60


MDP  medium
   policy_random:        211
   policy_earliest_free: 100
   policy_greedy:        66


MDP  large
   policy_random:        167
   policy_earliest_free: 100
   policy_greedy:        59


MDP  line
   policy_random:        82053
   policy_earliest_free: 85244
   policy_greedy:        48


MDP  complete
   policy_random:        156
   policy_earliest_free: 100
   policy_greedy:        85


MDP  nyc
   policy_random:        134
   policy_earliest_free: 70
   policy_greedy:        42




In [14]:
raw_csv_to_queries()

TypeError: raw_csv_to_queries() missing 1 required positional argument: 'path'